In [1]:
import pickle
with open('train.pkl', 'rb') as f:
    x, y = pickle.load(f)

print(x.shape, y.shape)  # (number_of_samples, height, width) (number_of_samples,)

(30000, 1, 150, 150) (30000,)


In [2]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=3)

In [3]:
import numpy as np

# Reshape X_train to the correct shape and duplicate the grayscale channel across 3 channels
X_train_rgb = np.repeat(x, 3, axis=1)

# Ensure the shape is now (24000, 3, 150, 150)
print(X_train_rgb.shape)

# If using TensorFlow, which expects the channel last format, you might need to transpose the axes
x = np.transpose(X_train_rgb, (0, 2, 3, 1))

# Ensure the shape is now (24000, 150, 150, 3) for TensorFlow
print(X_train_rgb.shape)

(30000, 3, 150, 150)
(30000, 3, 150, 150)


In [4]:
# Assuming 'y' is your array of labels, and it's one-hot encoded with a shape of (10000, 3)

# Convert one-hot encoded labels to integers for easy processing
y_integers = np.argmax(y, axis=1)

# Calculate the number of samples per class for an equal distribution
samples_per_class = 3333

# Initialize containers for the adjusted dataset
X_adjusted = np.empty((0, x.shape[1], x.shape[2], x.shape[3]))
y_adjusted = np.empty((0, y.shape[1]))

# Assuming there are exactly 3 classes
for class_index in range(3):
    # Find indices where the class label matches the current class index
    indices = np.where(y_integers == class_index)[0]
    
    # If the class has more samples than samples_per_class, randomly select samples_per_class samples
    if indices.size > samples_per_class:
        selected_indices = np.random.choice(indices, samples_per_class, replace=False)
    else:
        selected_indices = indices
    
    # Append the selected samples to the adjusted dataset
    X_adjusted = np.vstack((X_adjusted, x[selected_indices]))
    y_adjusted = np.vstack((y_adjusted, y[selected_indices]))

# Ensure the shapes of the adjusted datasets are correct
print(X_adjusted.shape)  # Should be close to (9999, 150, 150, 3) due to rounding
print(y_adjusted.shape)  # Should be close to (9999, 3) due to rounding


(9999, 150, 150, 3)
(9999, 3)


In [5]:
x = X_adjusted
y = y_adjusted

In [6]:
x.shape, y.shape

((9999, 150, 150, 3), (9999, 3))

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers

def build_augmented_multiclass_resnet50_model(input_shape=(150, 150, 3), num_classes=3):
    data_augmentation = tf.keras.Sequential([
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.2),
        layers.experimental.preprocessing.RandomZoom(0.2),
    ])
    
    base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)  # Apply data augmentation
    x = base_model(x, training=False)  # Base model in inference mode
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def build_augmented_binary_resnet50_model(input_shape=(150, 150, 3)):
    # Data augmentation layers
    data_augmentation = tf.keras.Sequential([
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.2),
        layers.experimental.preprocessing.RandomZoom(0.2),
    ])
    
    # Load pre-trained ResNet-50, exclude top layer
    base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    
    # New model on top
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)  # Apply data augmentation
    x = base_model(x, training=False)  # Base model in inference mode
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model


multiclass_model = build_augmented_multiclass_resnet50_model()
binary_model = build_augmented_binary_resnet50_model()


In [9]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [10]:
tf.debugging.set_log_device_placement(True)


In [11]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b_transposed = tf.transpose(b)  # Transpose b to make it [3, 2]
    c = tf.matmul(a, b_transposed)  # Now a is [2, 3] and b_transposed is [3, 2]

print(c)


tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [12]:
history = multiclass_model.fit(X_train, y_train,
                               validation_data=(X_val, y_val),
                               epochs=20,  # Adjust based on convergence
                               batch_size=200)  # Adjust based on your hardware


Epoch 1/20
40/40 [==============================] - 177s 4s/step - loss: 2.4170 - accuracy: 0.3284 - val_loss: 2.1594 - val_accuracy: 0.3235
Epoch 2/20
40/40 [==============================] - 166s 4s/step - loss: 2.1081 - accuracy: 0.3275 - val_loss: 1.9105 - val_accuracy: 0.3405
Epoch 3/20
40/40 [==============================] - 169s 4s/step - loss: 1.8697 - accuracy: 0.3298 - val_loss: 1.7257 - val_accuracy: 0.3345
Epoch 4/20
40/40 [==============================] - 170s 4s/step - loss: 1.6803 - accuracy: 0.3348 - val_loss: 1.5753 - val_accuracy: 0.3265
Epoch 5/20
40/40 [==============================] - 171s 4s/step - loss: 1.5478 - accuracy: 0.3339 - val_loss: 1.4646 - val_accuracy: 0.3415
Epoch 6/20
40/40 [==============================] - 168s 4s/step - loss: 1.4443 - accuracy: 0.3345 - val_loss: 1.3805 - val_accuracy: 0.3395
Epoch 7/20
40/40 [==============================] - 173s 4s/step - loss: 1.3639 - accuracy: 0.3345 - val_loss: 1.3151 - val_accuracy: 0.3245
Epoch 8/20
40

In [15]:
y.shape

(9999, 3)

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming x, y are your dataset and labels

# Define class pairs for binary classification
class_pairs = [
    (0, 1),  # "no" vs. "sub"
    (0, 2),  # "no" vs. "vertex"
    (1, 2)   # "sub" vs. "vertex"
]

binary_models = {}

for class_pair in class_pairs:
    # Convert one-hot encoded labels to class indices for filtering
    y_indices = np.argmax(y, axis=1)
    
    # Filter the dataset for the current pair of classes
    indices = np.where((y_indices == class_pair[0]) | (y_indices == class_pair[1]))[0]
    X_filtered = x[indices]
    y_filtered = y_indices[indices]
    
    # Create binary labels: 0 for the first class in the pair, 1 for the second
    y_binary = np.where(y_filtered == class_pair[0], 0, 1)
    
    # Split the filtered dataset
    X_train, X_val, y_train_binary, y_val_binary = train_test_split(X_filtered, y_binary, test_size=0.2, random_state=42)
    
    # Build and train the binary model for the current class pair
    binary_model = build_augmented_binary_resnet50_model(input_shape=(150, 150, 3))
    binary_model.fit(X_train, y_train_binary, validation_data=(X_val, y_val_binary), epochs=10, batch_size=32)
    
    # Store the trained model
    binary_models[class_pair] = binary_model


Epoch 1/10
167/167 [==============================] - 43s 215ms/step - loss: 0.7249 - accuracy: 0.5017 - auc: 0.5001 - val_loss: 0.6957 - val_accuracy: 0.4895 - val_auc: 0.4966
Epoch 2/10
167/167 [==============================] - 34s 205ms/step - loss: 0.7185 - accuracy: 0.4884 - auc: 0.4924 - val_loss: 0.7024 - val_accuracy: 0.4895 - val_auc: 0.4949
Epoch 3/10
167/167 [==============================] - 34s 206ms/step - loss: 0.7092 - accuracy: 0.5000 - auc: 0.4972 - val_loss: 0.6957 - val_accuracy: 0.4895 - val_auc: 0.5029
Epoch 4/10
167/167 [==============================] - 35s 208ms/step - loss: 0.7047 - accuracy: 0.4968 - auc: 0.5035 - val_loss: 0.6962 - val_accuracy: 0.4895 - val_auc: 0.4982
Epoch 5/10
167/167 [==============================] - 35s 208ms/step - loss: 0.7017 - accuracy: 0.4959 - auc: 0.4976 - val_loss: 0.6949 - val_accuracy: 0.4895 - val_auc: 0.5016
Epoch 6/10
167/167 [==============================] - 35s 208ms/step - loss: 0.7013 - accuracy: 0.4994 - auc: 0.495

In [17]:
def classify_image(image, multiclass_model, binary_models):
    # Predict with the multiclass model
    multiclass_pred = multiclass_model.predict(np.expand_dims(image, axis=0))
    
    # Find the top two classes
    top_two_classes = np.argsort(multiclass_pred[0])[-2:]
    
    # Map the indices to class pairs for the binary models dictionary
    class_pair = tuple(sorted(top_two_classes))
    
    # Use the selected binary model for final prediction
    binary_model = binary_models[class_pair]
    binary_pred = binary_model.predict(np.expand_dims(image, axis=0))
    
    # Assuming the binary classifier returns a probability for the class indexed by 1 in the sorted pair
    final_class_index = class_pair[1] if binary_pred >= 0.5 else class_pair[0]
    
    # Convert the final class index to a meaningful label
    class_labels = ['no', 'sub', 'vertex']
    final_prediction_label = class_labels[final_class_index]
    
    return final_prediction_label


In [36]:
X_val


array([[[[0.01084337, 0.01084337, 0.01084337],
         [0.01084337, 0.01084337, 0.01084337],
         [0.01204819, 0.01204819, 0.01204819],
         ...,
         [0.00722892, 0.00722892, 0.00722892],
         [0.0060241 , 0.0060241 , 0.0060241 ],
         [0.00722892, 0.00722892, 0.00722892]],

        [[0.01325301, 0.01325301, 0.01325301],
         [0.01084337, 0.01084337, 0.01084337],
         [0.01084337, 0.01084337, 0.01084337],
         ...,
         [0.01204819, 0.01204819, 0.01204819],
         [0.01325301, 0.01325301, 0.01325301],
         [0.00963855, 0.00963855, 0.00963855]],

        [[0.00843373, 0.00843373, 0.00843373],
         [0.01566265, 0.01566265, 0.01566265],
         [0.01204819, 0.01204819, 0.01204819],
         ...,
         [0.00843373, 0.00843373, 0.00843373],
         [0.01686747, 0.01686747, 0.01686747],
         [0.01445783, 0.01445783, 0.01445783]],

        ...,

        [[0.0060241 , 0.0060241 , 0.0060241 ],
         [0.00722892, 0.00722892, 0.00722892]

In [40]:
k = len(X_val)

In [42]:
# image = X_val[1500]  # Example image from the validation set

# Assuming 'multiclass_model' is your trained multiclass classifier
# and 'binary_models' is your dictionary of trained binary classifiers
sub = 0
vertex = 0
no = 0
for i in range(k):
    final_prediction_label = classify_image(X_val[i], multiclass_model, binary_models)
    if final_prediction_label == 'sub':
        sub += 1
    elif final_prediction_label == 'vertex':
        vertex += 1
    else:
        no += 1

1/1 [==============================] - 0s 23ms/step


In [43]:
no

0

In [44]:
vertex

1159

In [45]:
sub

175

In [54]:
y_val

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [56]:
import os
def load_data_and_labels():
    x = []
    y = []

    # Assign labels to each class
    labels = {"no": 0, "sphere": 1, "vort": 2}

    for class_dir, label in labels.items():
        npy_files = os.listdir("val/{}".format(class_dir))  # List all files in the class directory
        for file in npy_files:
            file_path = os.path.join("val/{}".format(class_dir), file)
            data = np.load(file_path)
            x.append(data)
            y.append(label)

    return np.array(x), np.array(y)

# Assuming 'dataset_path' is the path to your 'dataset' directory
# class_dirs = {"train/no": 0, "train/sphere": 1, "train/vort": 2}
x, y = load_data_and_labels()


In [57]:
import numpy as np

# Reshape X_train to the correct shape and duplicate the grayscale channel across 3 channels
X_train_rgb = np.repeat(x, 3, axis=1)

# Ensure the shape is now (24000, 3, 150, 150)
print(X_train_rgb.shape)

# If using TensorFlow, which expects the channel last format, you might need to transpose the axes
x = np.transpose(X_train_rgb, (0, 2, 3, 1))

# Ensure the shape is now (24000, 150, 150, 3) for TensorFlow
print(X_train_rgb.shape)

(7500, 3, 150, 150)
(7500, 3, 150, 150)


In [58]:
x.shape

(7500, 150, 150, 3)

In [59]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=3)

In [62]:
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy('mixed_float16')


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Laptop GPU, compute capability 8.6


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Laptop GPU, compute capability 8.6


In [70]:
multiclass_loss, multiclass_accuracy = multiclass_model.evaluate(x[1000:7000], y[1000:7000])
print(f"Multiclass Test Loss: {multiclass_loss}")
print(f"Multiclass Test Accuracy: {multiclass_accuracy}")

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [61]:
# Saving the multiclass model
multiclass_model.save('multiclass_model')

# Saving binary models
for class_pair, binary_model in binary_models.items():
    binary_model.save(f'binary_model_{class_pair[0]}_{class_pair[1]}')


INFO:tensorflow:Assets written to: multiclass_model\assets


INFO:tensorflow:Assets written to: multiclass_model\assets


INFO:tensorflow:Assets written to: binary_model_0_1\assets


INFO:tensorflow:Assets written to: binary_model_0_1\assets


INFO:tensorflow:Assets written to: binary_model_0_2\assets


INFO:tensorflow:Assets written to: binary_model_0_2\assets


INFO:tensorflow:Assets written to: binary_model_1_2\assets


INFO:tensorflow:Assets written to: binary_model_1_2\assets
